In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import os
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Subset
import numpy as np

In [23]:
data = ImageFolder("data/", transform=transform)

class_indices = {i: np.where(np.array(data.targets) == i)[0] for i in range(len(data.classes))}
subset_indices = []

for indices in class_indices.values():
    subset_size = int(0.1 * len(indices))
    selected_indices = np.random.choice(indices, subset_size, replace=False)
    subset_indices.extend(selected_indices)

subset_data = Subset(data, subset_indices)

train_size = int(0.8 * len(subset_data))
val_size = int(0.1 * len(subset_data))
test_size = len(subset_data) - train_size - val_size

train_data, val_data, test_data = random_split(subset_data, [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [25]:
tensorboardWriter = SummaryWriter('logs/experiment_1')